# Week 7 – Feature Engineering with Pipelines

>## Learning goals
>
>By the end of this notebook, you should be able to:
>
> - Review how **preprocessing pipelines** work in scikit-learn.  
> - Load and inspect a **real-world movies dataset** with mixed data types.  
> - Handle **time-based features**.
> - Implement custom cleaning and feature engineering using **`FunctionTransformer`**.  
> - Combine **numeric** and **categorical** preprocessing with `ColumnTransformer`.  
> - Build a **full preprocessing pipeline** using `make_pipeline`.
> - Export a fully cleaned and transformed dataset for future use.
___
**Install the required packages**

In [ ]:
# uncomment the following line to install the required package
#!pip install IPython
#!pip install scikit-learn

**Import necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

**Import datetime to handle time data**

In [ ]:
import datetime as dt

**Import scikit-learn preprocessing pipelines functions**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.utils import estimator_html_repr

## 1. Review of Last Class — Preprocessing Pipelines

In the previous class, you:

> - Cleaned the **Adult Income** dataset manually.  
> - Recreated the same logic with **scikit-learn pipelines**.  
> - Built separate pipelines for **numeric** and **categorical** features.  
> - Combined them with `ColumnTransformer`.  
> - Used `set_output(transform="pandas")` to get back a DataFrame.  
> - Exported the cleaned dataset to `.csv`.
>
>**Why was that important?
>
> - The preprocessing steps became **reproducible and consistent**.  
> - You could apply exactly the same transformations to **new data**.  
> - All cleaning logic was concentrated in a **single object** (the pipeline), instead of scattered across many cells.


## 2. The Movies Dataset
>
>In this class, we will work with a subset of **The Movies Dataset** (Kaggle).  
>We will use the file **`movies_dataset.csv`**, which contains thousands of movies with:
>
> - Titles, languages, and textual descriptions  
> - Budget, revenue, popularity, runtime  
> - `release_date` (temporal information!)  
> - Categorical attributes like `status`, `original_language`, `adult`  
> - Some JSON-like fields such as `genres`
>
>**For now, we will focus on:**
>
> - Cleaning a subset of relevant columns  
> - Converting `release_date` into useful features  
> - Building a preprocessing pipeline around this.
___
### Q2.1. Load the dataset and show the first rows
> - use `pd.read_csv("movies_metadata.csv")`.  
> - store the result in a DataFrame named `df`.  
> - display the **first 5 rows** with `df.head()`.

In [ ]:
# your code here


### Q2.2. Inspect the basic structure of the dataset
>
> - Check the **shape** → number of rows and columns.  
> - Use `df.info()` to inspect data types and non-null counts.  
> - Use `df.describe()` to inspect basic statistics.
>
>**This helps to identify:**
>
> - which columns are numeric  
> - which columns are objects / strings  
> - where there are missing values or strange types

In [ ]:
# your code here


In [ ]:
# your code here


### Q2.3. Create a list of categorical features

In [ ]:
# your code here


### Q2.4. Create a list of numeric features

In [ ]:
# your code here


### Q2.5. Inspect the different values of categorical features
>- use the list of categoricals that you have created earlier
>- use `value_count()` or `.unique()`

In [ ]:
# your code here


### Focus for this class
>
>**We will focus on the following columns:**
>
> - **`title`** – movie title (string)  
> - **`original_language`** – language code (categorical)  
> - **`adult`** – whether the movie is for adults (`'True'` / `'False'`)  
> - **`status`** – release status (`Released`, `Rumored`, etc.)  
> - **`release_date`** – when the movie was released  
> - **`budget`** – production budget (numeric, but sometimes `0` or invalid)  
> - **`revenue`** – total revenue (same issues as budget)  
> - **`runtime`** – duration in minutes  
> - **`vote_average`**, **`vote_count`**, **`popularity`** – numeric indicators  
>
>**We will build a pipeline that:**
>
> - Cleans and converts these columns.  
> - Derives time-based features.  
> - Prepares everything for downstream ML tasks.
___

### Q2.6. Explore missing values and strange entries

> - Compute the **percentage of missing values** per column
> - Inspect some rows where `release_date` is missing or invalid.  

This will help you understand **why** we need custom cleaning logic.


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


## 3. Custom Cleaning Function for Movies

We will now write a **custom cleaning function** that:

> - Converts `release_date` to a proper `datetime` (`errors="coerce"`).  
> - Converts `budget` and `revenue` to numeric using `pd.to_numeric(..., errors="coerce")`.  
> - Replaces **0 values** in `budget` and `revenue` with `NaN` (they usually mean "unknown").  
> - Ensures `adult` is standardized as a simple `'True'` / `'False'` string or boolean.  

This function will **not** be called directly on `df`.  
Instead, we will later wrap it in a **`FunctionTransformer`** to plug it into a pipeline.

### Q3.1. Implement the `basic_movies_cleaning()` function


In [ ]:
import numpy as np

def basic_movies_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a cleaned copy of the movies metadata DataFrame.

    Steps:
    - Convert 'release_date' to datetime (errors='coerce').
    - Convert 'budget' and 'revenue' to numeric (errors='coerce').
    - Replace 0 values in 'budget' and 'revenue' with NaN.
    """
    df_new = df.copy()

    # Convert release_date to datetime
    

    # Convert budget and revenue to numeric
    

    # Replace 0 with NaN for budget and revenue
    

    return df_new

### Q3.2. Test `basic_movies_cleaning` on a small sample
> - Apply the function to a sample using `.sample()` and set `random_state=42`
> - Call `basic_movies_cleaning(df_sample)` and inspect the result.  
> - Check that:
>   - `release_date` is now `datetime64[...]`  
>   - `budget` and `revenue` are numeric  
>   - 0 values in `budget` / `revenue` were replaced by NaN


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


## 4. Time-based Feature Engineering
>We are going to work now on time features.
>
>The `release_date` is a good feature that can provide us a lot of information
>
> Example of **time-based features**, that can be extracted:
>
> - `release_year`  
> - `release_month`  
> - `release_day`  
> - `release_weekday` (0 = Monday, 6 = Sunday)  
> - `movie_age` (e.g. difference between current year and release year)  
>
>We will implement this in a separate function.  
>Later, both functions will be combined inside a **pipeline**.
___
### Q4.2. First convert `release_date` column to datetime
- use `pd.to_datetime(df['release_date'], errors='coerce')`, you can specify the `format='%Y-%m-%d'`

In [ ]:
# your code here


In [ ]:
# your code here


### Q4.3. Explore some datetime attributes
- use `dt.year`, `dt.month`, `dt.day` and `dt.weekday`
- inspect weekday closely

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


**Noticed that weekday starts on 0 (Monday) and ends on 6 (Sunday)?**
___
### Q4.4. Let's do some datetime comparinsons
>- a) Verify for the movies within the minimum, mean, median and maximum date
>- b) Verify for the oldest and most recent movies in the dataset
>- c) Verify what `dt.datetime.now()` will print as output, then check it out with `Q4.3 attributes`
>    - **Note:** use `.weekday()` instead of `.weekday`
>- d) Verify for the difference of the current year and the `release_date` year
>    - **Note:** what would that difference correspond to

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


### Q4.5. Implement `add_time_features()` function
>```python
>   def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
>        """
>        Receives a DataFrame with a 'release_date' column (datetime)
>        and returns a copy with new time-based features:
>        
>        - release_year
>        - release_month
>        - release_day
>        - release_weekday
>        - movie_age (assuming reference year = current year)
>        """
>        df_new = df.copy()
>        df_new.rename(columns={'cat__release_date': 'release_date'}, inplace=True)
>
>        # your code here
>
>        return df_new

In [ ]:
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Receives a DataFrame with a 'release_date' column (datetime)
    and returns a copy with new time-based features:
    
    - release_year
    - release_month
    - release_day
    - release_weekday
    - movie_age (assuming reference year = current year)
    """
    df_new = df.copy()
    df_new.rename(columns={'cat__release_date': 'release_date'}, inplace=True)


    # your code here


    return df_new


### Q4.6. Test `add_time_features` after cleaning

> - Take a small sample using `.sample()`
> - First apply `basic_movies_cleaning`  
> - Then apply `add_time_features`  
> - Inspect the new columns using this subset `["title", "release_date", "release_year", "release_month", "release_weekday", "movie_age"]`


In [ ]:
# print subset ["title", "release_date", "release_year", "release_month", "release_weekday", "movie_age"]

In [ ]:
# your code here


**We are still on the surface of what we can do with `datetime` columns**
- We are going to change the column back to object just to make it as it was before we started playing a bit with some functions
- Remember that we are going to handle the conversion of this column in our pipeline
___
### Q4.6. Cast the `release_date` column back to object
>- use `.astype(str)`

In [ ]:
# your code here


### 5. Recap
>
>Up until now, what have we done?
>
>It seems huge because we gradually built our workflow...
___
### Q5.1. Take this moment to look back and summarize in steps what we have accomplished so far

your answer here

## 6. Choosing Numeric and Categorical Features

We now decide which columns will be treated as **numeric** and which as **categorical** for preprocessing.

For this class, we will:

> - Select the following column from **categorical** list for preprocessing:
>   - `status`  
> - Consider all remaining **non-datetime** numeric columns as **numeric features**.  
> - Ignore long text fields like `overview`, `tagline` for now.

### Q6.1.Select the features for preprocessing
> - create the `encode_features` list
> - remove `release_date` if present (datetime) in numeric or categorical list

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


## 7. Numeric and Categorical Preprocessing Pipelines

We now define:

> - `numeric_pipeline`
>   - Imputes missing values with the **median**.  
>   - Scales features using `StandardScaler`.  
>
> - `categorical_pipeline`
>   - Imputes missing values with the most frequent category.
>
> - `encode_pipeline`
>   - Imputes missing values with the most frequent category.
>   - Encodes categories using `OneHotEncoder` with `handle_unknown="ignore"`.
___
### Q7.1. Implement `numeric_pipeline`, `categorical_pipeline` and `encode_pipeline`
- use the following:
>```python
>numeric_pipeline = Pipeline([
>    ("imputer", SimpleImputer(strategy="median")),
>    ("scaler", StandardScaler()),
>])
>
>categorical_pipeline = Pipeline([
>    ("imputer", SimpleImputer(strategy="most_frequent")),
>])
>
>encode_pipeline = Pipeline([
>    ("imputer", SimpleImputer(strategy="most_frequent")),
>    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
>])
>```

In [ ]:
# your code here


> **Note on One-Hot Encoding Output**
>
> By default, `OneHotEncoder` can return a **sparse matrix**.
> In this notebook, we set `sparse_output=False` so that the output is a **dense array**.
>
>   **Example of Dense Output:**
>```console
>                language_en  language_fr  language_pt
>           0    1            0            0
>           1    0            1            0
>           2    0            0            1
>```
>   - Dense arrays are easier to convert into pandas DataFrames.
>
>   **Example of Sparse Output:**
>```console
>           (0, 0)	1
>           (1, 1)	1
>           (2, 2)	1
>```
>   - Each row shows:
>       - the **position** `(row_index, column_index)`  
>       - the **value** at that position  
>       - all other positions are implicitly **zero**.


## 8. Combine Everything with `ColumnTransformer`

We now combine the two preprocessing pipelines into a single `ColumnTransformer` that:

> - Applies `numeric_pipeline` to `numeric_features`  
> - Applies `categorical_pipeline` to `categorical_features`  
> - Drops remaining columns (the default behavior)
>
>**Example:**
>```python
>   full_preprocessor = ColumnTransformer(
>        transformers=[
>            ("num", numeric_pipeline, numeric_features),
>            ("cat", categorical_pipeline, categorical_features),
>            ("enc", encode_pipeline, encode_features),
>        ],
>        remainder="passthrough",
>        verbose_feature_names_out=False
>    )
>```
>
>**Note 1: The parameter `remainder="passthrough"` will keep the new columns generated and make them passthrough the rest of the pipeline** 
>**Note 2: The parameter `verbose_feature_names_out=False` will mantain the original name of the columns** 
___
### Q8.1. Create `full_preprocessor`



In [ ]:
# your code here


## 9. Full Movies Preprocessing Pipeline

We can now create a **single pipeline** that:

> 1. Applies `basic_movies_cleaning`  
> 2. Applies `add_time_features`  
> 3. Applies `full_preprocessor` (numeric + categorical + encode)

We will use `make_pipeline` again and finally call `set_output(transform="pandas")` so that the final result is a **DataFrame**, not a NumPy array.

**Example:**
>```python
>    full_pipeline = make_pipeline(
>        FunctionTransformer(basic_movies_cleaning, validate=False),
>        FunctionTransformer(add_time_features, validate=False),
>        full_preprocessor,
>    )
>
>    full_pipeline.set_output(transform="pandas")
>```
___
### Q9.1. Create `full_pipeline`


In [ ]:
# your code here


## 10. Visualizing the Preprocessing Pipeline (Diagram)

Scikit-learn can generate an **HTML diagram** of the entire pipeline using:

> - `sklearn.utils.estimator_html_repr`  
> - `IPython.display.HTML`

This is extremely useful to **inspect the structure** of your preprocessing workflow.

### Q10.1. Generate the HTML representation of `full_pipeline`
>
> ⚠️ The diagram may not render outside Jupyter, but should work in a classic notebook environment.


In [ ]:
HTML(estimator_html_repr(full_pipeline))

## 11. Apply the Full Pipeline to the Movies Dataset

We now apply all steps in a **single call**:

> - custom cleaning  
> - time-based feature engineering  
> - numeric, categorical and encoding preprocessing
___
### Q11.1. Apply `full_pipeline` to `df` and inspect the result
> - use `fullpipeline.fit_transform(df)`
> - Store the result in `df_clean`.  
> - Show the first 5 rows.  
> - Print the final shape (rows, columns).

In [ ]:
# your code here


In [ ]:
# your code here


In [ ]:
# your code here


## 12. Export the Cleaned Movies Dataset

Finally, we export the processed dataset to a `.csv` file so that it can be reused.

### Q12.1. Save `df_clean` as `clean_movies_preprocessed.csv` (no index).


In [ ]:
# your code here


---

## 13. Extra Exercises

> These are optional but recommended exercises if you want to deepen your understanding.

### Q13.1. Create a `profit` and `roi` feature

> - `profit = revenue - budget`  
>
> - **roi == Return of Investment**
>   - `roi = revenue / budget` (be careful with division by zero / NaN)

In [ ]:
# your code here


### Q13.2. Verify the number of movies released per year and plot
- use `.groupby('release_year').size()`
- use `.plot(kind='line')`

In [ ]:
# your code here


### Q13.3. Verify the average budget per year and plot
- use `.groupby("release_year")["budget"].mean()`
- use `.plot(kind='line')`

In [ ]:
# your code here


### Q13.4. Verify Monthly distribution of releases and plot
- use `df_clean.groupby("release_month").size()`
- use `.plot(kind='line')`

In [ ]:
# your code here


### Q13.5. Verify movies released by weekday and plot

In [ ]:
# your code here


### Q13.6. Verify the average popularity by movie age and plot
- use `.groupby("movie_age")["popularity"].mean()`
- use `.plot(kind='line')`

In [ ]:
# your code here


### 🎬🍿 After those exercises you have earned the right to watch some movies!
>
>**Whether you prefer:**
>🎭 drama
>🚀 sci-fi
>😂 comedy
>🦸 action
>🧙‍♂️✨ fantasy
>💘 romance
>
>…go treat yourself to a well-earned film session! 🎞️🍫